<a href="https://colab.research.google.com/github/muraliRoyal1/venkatasiva_INFO5731_Fall2020/blob/master/Muralli_In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
# Write your code here

import pandas as pd
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dd = pd.read_csv("/content/sample_data/murallidata.csv")
dd = dd[['abs']]
dd['abs'] = dd['abs'].str.replace('[^\w\s]','')
dd['abs'] = dd['abs'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dd['abs'] = dd['abs'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
dd['abs'] = dd['abs'].apply(lambda x: nltk.word_tokenize(x))
values = (dd['abs']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index()
values.columns = ['Words_List', 'tf']
values['polarity_Value'] = values['Words_List'].apply(lambda x: TextBlob(x).sentiment.polarity)
terms = values.loc[values['polarity_Value'] != 0].sort_values(by='tf', ascending=False)
terms = terms.reset_index(drop=True)
terms.head(50)

,Words_List,tf,polarity_Value
0,natural,71.0,0.100000
1,linguistic,10.0,0.100000
2,new,8.0,0.136364
3,many,7.0,0.500000
4,effective,7.0,0.600000
5,developed,5.0,0.100000
6,significant,4.0,0.375000
7,limited,4.0,-0.071429
8,general,4.0,0.050000
9,wide,4.0,-0.100000


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:

from sklearn.metrics import f1_score, accuracy_score
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

dd1 = pd.read_csv("/content/sample_data/murallidata1 .csv")
dd1['polarity'] = dd1['abs'].apply(lambda x: TextBlob(x).sentiment.polarity)
dd1['pred sen'] = pd.cut(dd1['polarity'], bins=3, labels=[1, 2, 3])
def sentiment(x):
    if x == 1:
        return 'Positive'
    if x == 2:
        return 'Neutral'
    if x ==3:
        return 'Negative'
dd1['pred sen'] = dd1['pred sen'].apply(lambda x: sentiment(x))
print(dd1[[ 'sen', 'pred sen']].head(50))
textblob_accuracy = accuracy_score(dd1['sen'], dd1['pred sen'])*100
textblob_f1 = f1_score(dd1['sen'], dd1['pred sen'], average='macro')
vader = SentimentIntensityAnalyzer()

dd3 = pd.read_csv("/content/sample_data/murallidata1 .csv")
dd3['polarity'] = dd1['abs'].apply(lambda x: vader.polarity_scores(x)['compound'])
dd3['pred sen'] = pd.cut(dd3['polarity'], bins=3, labels=[1, 2, 3])

dd3['pred sen'] = dd3['pred sen'].apply(lambda x: sentiment(x))
print(dd3[['sen', 'pred sen']].head(50))

vader_accuracy = accuracy_score(dd3['sen'], dd3['pred sen'])*100
vader_f1 = f1_score(dd3['sen'], dd3['pred sen'], average='macro')

dd4 = pd.read_csv("/content/sample_data/murallidata1 .csv")
train, test = sklearn.model_selection.train_test_split(dd4, train_size=0.5, test_size=0.2)
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])
svm = pipeline.fit(train['abs'], train['sen'])
test['pred sen'] = svm.predict(test['abs'])
print(test[[ 'sen', 'pred sen']].head(50))
svm_accuracy = accuracy_score(test['sen'], test['pred sen'])*100
svm_f1 = f1_score(test['sen'], test['pred sen'], average='macro')
print( svm_f1)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
         sen  pred sen
0   positive   Neutral
1   positive   Neutral
2   positive   Neutral
3   positive  Negative
4   positive  Negative
5   positive   Neutral
6   positive   Neutral
7   positive   Neutral
8   positive   Neutral
9   positive   Neutral
10  positive   Neutral
11  positive  Negative
12  positive  Negative
13  positive  Negative
14  positive   Neutral
15  positive   Neutral
16  negative  Negative
17  positive   Neutral
18  positive   Neutral
19  positive   Neutral
20  negative  Negative
21  positive  Negative
22  positive   Neutral
23  positive   Neutral
24  positive   Neutral
25  positive   Neutral
26  positive   Neutral
27  positive   Neutral
28  positive   Neutral
29  positive   Neutral
30  positive  Negative
31  negative   Neutral
32  positive   Neutral
33  positive   Neutral
34  positive   Neutral
35  positive   Neutral
36  positive   Neutral

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
"""from above analysis out of all models svm resulted in better for sentiment analysis with 0.45 f1 score